In [1]:
import os
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from model_settings import ms
from plotters import PlotCols

ms.find_root(Path())


pd.set_option('display.max_columns',None)

def HistCols(df,columns=None,sharex=True):
    if columns == None:
        columns = df.columns.tolist()
        sharex = False

    fig,axs=plt.subplots(len(columns),sharex=sharex,figsize=(10,20))
    for i,col in enumerate(columns):
        axs[i].hist(df[col],label=col.replace('_',' '),color='purple',bins=int(np.sqrt(df.shape[0])))
        axs[i].legend()

def test_model(df,pricename):
    df = df.copy()
    df['target_price'] = df[pricename]
    df['observed_price'] = df['relative_observed']*df['strike_price']
    df['pricing_error'] = df['observed_price']-df[pricename]
    df['relative_error'] = df['pricing_error']/df[pricename]
    df['squared_error'] = df['pricing_error']**2
    df['absolute_error'] = np.abs(df['pricing_error'])
    df['absolute_relative'] = np.abs(df['relative_error'])
    desc = df.describe()
    MAE = desc.loc['mean','absolute_error']
    RMSE = np.sqrt(desc.loc['mean','squared_error'])
    MARE = desc.loc['mean','absolute_relative']
    print(f"MAE:\n   {round(MAE,4)}\nRMSE:\n   {round(RMSE,4)}\nMARE:\n   {round(100*MARE,4)}%")
    return df.iloc[:,-7:]

In [ ]:
"""
NEED DATA FIRST
"""

In [6]:
errors = model['errors']
models = model['models']

KeyError: 'errors'

In [ ]:
PlotCols(errors)

In [ ]:
dates = pd.Series([d for d in models.keys()])
dates.describe()[['min','max']]

In [ ]:
model = models[dates.iloc[-1]]
data = model['test_data']
pricename = [n for n in data.columns if n.find('barrier_price')!=-1 or n.find('asian_price')!=-1][0]

In [ ]:
df = test_model(data,pricename)
df.columns

In [ ]:
HistCols(df,sharex=False)

In [ ]:
estimation = df['observed_price']
target = df['target_price']
relative = df['relative_error']

In [ ]:
for k in model.keys():
    print(k)

In [ ]:
trained = model['model']
print(trained)

In [ ]:
params = model['test_data'][['calculation_date','kappa','theta','eta','rho','v0']].copy().set_index('calculation_date').drop_duplicates(keep='last')

In [ ]:
import pandas as pd
import numpy as np
from itertools import product
import quantlib_pricers as qlp

s = [100]
K = [80,100,120]
T = [30,60,90]

features_dict = {
    'spot_price':s,
    'strike_price':K,
    'days_to_maturity':T,
    'risk_free_rate':[0.04],
    'dividend_rate':[0.00],
    'w':['call','put'],
    'barrier_type_name':['DownOut'],
    'rebate':[0],
    'barrier':[50,100,150],
}

def CartesianProduct(features_dict):
    return pd.DataFrame(product(*features_dict.values()),columns=features_dict.keys())

In [ ]:
test_data = model['test_data'].copy()